In [35]:
import requests
import urllib
import folium
import gspread
import pandas as pd
from folium import plugins
import branca
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import os
import geopandas as gpd

In [36]:
# 認証情報の取得
scope = ['https://www.googleapis.com/auth/spreadsheets']
credentials = service_account.Credentials.from_service_account_file('keys.json', scopes=scope)

In [37]:
# スプレッドシートにアクセス
gClient = gspread.authorize(credentials)
SPREADSHEET_KEY = '11ajawzcNAp2hoq5897aaP2kCBYw2I7ZEDB0CK-mNapQ'
file = gClient.open_by_key(SPREADSHEET_KEY)

In [42]:
# シートにアクセス
sheet1 = file.worksheet('新築戸建て')

# sheet1のデータフレームを作成
data_sheet1 = sheet1.get_all_values()
df_sheet1 = pd.DataFrame(data_sheet1[1:], columns=data_sheet1[0])

#新築戸建ての緯度経度取得
latitude_list = [] 
longtude_list = []

name_list = df_sheet1["物件名"].tolist()
price_list = df_sheet1["販売価格"].tolist() 
plan_list = df_sheet1["間取り"].tolist()
area_list = df_sheet1["面積"].tolist() 
URL_list = df_sheet1["URL"].tolist()
address_list = df_sheet1["所在地"].tolist() 

url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

for i in range(len(df_sheet1)):
    q = df_sheet1["所在地"][i]
    res = requests.get(url + urllib.parse.quote(q))

    latitude_list.append(res.json()[0]["geometry"]["coordinates"][1]) 
    longtude_list.append(res.json()[0]["geometry"]["coordinates"][0])

#マップにプロット
map = folium.Map(location=[latitude_list[0],longtude_list[0]],zoom_start=15)

for i in range(len(df_sheet1)):
    popup_content = f"<div style='writing-mode: horizontal-tb; white-space: nowrap;'>"
    popup_content += f"{name_list[i]}<br>"
    popup_content += f"販売価格: {price_list[i]}<br>"
    popup_content += f"間取り: {plan_list[i]}<br>"
    popup_content += f"面積: {area_list[i]}<br>"
    popup_content += f'<a href={URL_list[i]} target="_blank">詳細</a>' 
    popup_content += "</div>"
    
    iframe = branca.element.IFrame(html=popup_content, width=300, height=160)
    popup = folium.Popup(iframe, max_width=300)

    folium.Marker(location=[latitude_list[i], longtude_list[i]], popup=popup,icon=folium.Icon(color='orange')).add_to(map)

map


In [43]:
df_sheet1

,物件名,販売価格,販売価格2,所在地,沿線・駅,土地面積,間取り,間取り2,面積,面積2,URL
0,【オープンハウスグループ】ミラスモシリーズ横浜市都筑…,3980万円,3980,神奈川県横浜市都筑区川和町,グリーンライン「川和町」徒歩15分,72.95m2,1LDK+S（納戸）,1,69.14m2,69.14,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...
1,オープンライブス川和町スカイステージ,3980万円,3980,神奈川県横浜市都筑区川和町,グリーンライン「川和町」徒歩13分,46.29m2,2LDK+S（納戸）,2,82.37m2,82.37,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...
2,神奈川県横浜市都筑区川和町 新築戸建,4180万円,4180,神奈川県横浜市都筑区川和町,グリーンライン「川和町」徒歩15分,71.5m2（登記）,1LDK+S（納戸）,1,69.14m2,69.14,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...
3,【安心を買うなら、朝日土地建物へ】都筑区早渕３丁目 …,4280万円～4380万円,4280～4380,神奈川県横浜市都筑区早渕３,グリーンライン「東山田」徒歩8分,45.93m2～46.19m2（登記）,3LDK,3,83.03m2～91.91m2（登記）,83.03,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...
4,新築分譲住宅 全３邸 都筑区早渕３丁目,4280万円～4380万円,4280～4380,神奈川県横浜市都筑区早渕３,グリーンライン「東山田」徒歩8分,45.93m2～46.19m2（13.89坪～13.97坪）（登記）,3LDK,3,83.03m2～91.91m2（25.11坪～27.80坪）（登記）,83.03,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...
5,横浜市都筑区南山田町 新築戸建1棟,4480万円,4480,神奈川県横浜市都筑区南山田町,グリーンライン「東山田」徒歩14分,71.05m2,2LDK,2,58.82m2、うち１階車庫2.21m2,58.82,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...
6,都筑区川和町 新築戸建全６邸,5580万円～5780万円,5580～5780,神奈川県横浜市都筑区川和町2383,グリーンライン「川和町」徒歩9分,165.18m2～173.41m2（49.96坪～52.45坪）（実測）,3LDK+S（納戸）～4LDK,3,104.85m2～106.19m2（31.71坪～32.12坪）（実測）,104.85,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...
7,,6297万5000円・6597万5000円,6297.5・6597.5,神奈川県横浜市都筑区茅ケ崎南３,ブルーライン「センター南」徒歩13分～14分,66.88m2～67.36m2,1LDK+3S（納戸）・2LDK+2S（納戸）,1,127.03m2～130.74m2,127.03,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...
8,//約51坪・南西角地の開放感を存分に♪//並列2台…,7180万円,7180,神奈川県横浜市都筑区南山田町,グリーンライン「北山田」徒歩10分,170.81m2（51.66坪）,4LDK+S（納戸）,4,113.44m2（34.31坪）,113.44,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...
9,LAGRAS荏田南第1期,7780万円～8480万円,7780～8480,神奈川県横浜市都筑区荏田南３,東急田園都市線「市が尾」徒歩18分,165.48m2～195.62m2（実測）,4LDK,4,126.96m2～133.61m2,126.96,https://suumo.jp/ikkodate/kanagawa/sc_yokohama...


In [39]:
# シートにアクセス
sheet2 = file.worksheet('中古戸建て')

# sheet2のデータフレームを作成
data_sheet2 = sheet2.get_all_values()
df_sheet2 = pd.DataFrame(data_sheet2[1:], columns=data_sheet2[0])

#中古戸建ての緯度経度取得
latitude_list = [] 
longtude_list = []

name_list = df_sheet2["物件名"].tolist()
price_list = df_sheet2["販売価格"].tolist() 
plan_list = df_sheet2["間取り"].tolist()
area_list = df_sheet2["面積"].tolist() 
URL_list = df_sheet2["URL"].tolist()
address_list = df_sheet2["所在地"].tolist() 

url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

for i in range(len(df_sheet2)):
    q = df_sheet2["所在地"][i]
    res = requests.get(url + urllib.parse.quote(q))

    latitude_list.append(res.json()[0]["geometry"]["coordinates"][1]) 
    longtude_list.append(res.json()[0]["geometry"]["coordinates"][0])

#マップにプロット
map = folium.Map(location=[latitude_list[0],longtude_list[0]],zoom_start=15)

for i in range(len(df_sheet2)):
    popup_content = f"<div style='writing-mode: horizontal-tb; white-space: nowrap;'>"
    popup_content += f"{name_list[i]}<br>"
    popup_content += f"販売価格: {price_list[i]}<br>"
    popup_content += f"間取り: {plan_list[i]}<br>"
    popup_content += f"建物面積: {area_list[i]}<br>"
    popup_content += f'<a href={URL_list[i]} target="_blank">詳細</a>' 
    popup_content += "</div>"
    
    iframe = branca.element.IFrame(html=popup_content, width=300, height=160)
    popup = folium.Popup(iframe, max_width=300)

    folium.Marker(location=[latitude_list[i], longtude_list[i]], popup=popup,icon=folium.Icon(color='green')).add_to(map)

map

In [44]:
df_sheet2

,物件名,販売価格,販売価格2,所在地,沿線・駅,土地面積,間取り,間取り2,面積,面積2,URL
0,【安心を買うなら、朝日土地建物へ】都筑区佐江戸町 中…,3550万円,3550,神奈川県横浜市都筑区佐江戸町,ＪＲ横浜線「中山」徒歩20分,71.21m2（登記）,2LDK+S（納戸）,2,93.67m2（登記）,93.67,https://suumo.jp/chukoikkodate/kanagawa/sc_yok...
1,◆眺望も満喫できる家 リノベーション戸建 見学予約受…,3550万円,3550,神奈川県横浜市都筑区佐江戸町,ＪＲ横浜線「中山」徒歩20分,71.21m2（登記）,2LDK+S（納戸）,2,93.67m2（登記）,93.67,https://suumo.jp/chukoikkodate/kanagawa/sc_yok...
2,加賀原1丁目中古戸建,6580万円,6580,神奈川県横浜市都筑区加賀原１,グリーンライン「川和町」徒歩12分,166.44m2（50.34坪）（登記）,3LDK+S（納戸）,3,105.58m2（31.93坪）（登記）,105.58,https://suumo.jp/chukoikkodate/kanagawa/sc_yok...
3,,6580万円,6580,神奈川県横浜市都筑区加賀原１,グリーンライン「川和町」徒歩12分,166.44m2,3LDK+S（納戸）,3,105.58m2,105.58,https://suumo.jp/chukoikkodate/kanagawa/sc_yok...
4,【安心を買うなら、朝日土地建物へ】都筑区川和町 中古…,6980万円,6980,神奈川県横浜市都筑区川和町,グリーンライン「川和町」徒歩8分,286.76m2（登記）,7LDK+2S（納戸）,7,220.68m2（登記）,220.68,https://suumo.jp/chukoikkodate/kanagawa/sc_yok...
...,...,...,...,...,...,...,...,...,...,...,...
85,,1億4900万円,14900,神奈川県横浜市都筑区高山,グリーンライン「都筑ふれあいの丘」徒歩3分,313.78m2（94.91坪）（登記）,2LDK,2,154.66m2（46.78坪）（登記）,154.66,https://suumo.jp/chukoikkodate/kanagawa/sc_yok...
86,,1億5400万円,15400,神奈川県横浜市都筑区牛久保２,ブルーライン「中川」徒歩10分,238.92m2（72.27坪）（登記）,4LDK+3S（納戸）,4,161.75m2（48.92坪）（登記）,161.75,https://suumo.jp/chukoikkodate/kanagawa/sc_yok...
87,,1億5800万円,15800,神奈川県横浜市都筑区茅ケ崎南１,ブルーライン「センター南」徒歩13分,261.89m2（登記）,6LDK+S（納戸）,6,156.27m2（登記）,156.27,https://suumo.jp/chukoikkodate/kanagawa/sc_yok...
88,,1億5800万円,15800,神奈川県横浜市都筑区仲町台３,ブルーライン「仲町台」徒歩12分,384.46m2（登記）,8LDK,8,262.09m2,262.09,https://suumo.jp/chukoikkodate/kanagawa/sc_yok...


In [40]:
sheet3 = file.worksheet('中古マンション')

# sheet3のデータフレームを作成
data_sheet3 = sheet3.get_all_values()
df_sheet3 = pd.DataFrame(data_sheet3[1:], columns=data_sheet3[0])

#中古マンションの緯度経度取得
latitude_list = [] 
longtude_list = []

name_list = df_sheet3["物件名"].tolist()
price_list = df_sheet3["販売価格"].tolist() 
plan_list = df_sheet3["間取り"].tolist()
area_list = df_sheet3["面積"].tolist() 
URL_list = df_sheet3["URL"].tolist()
address_list = df_sheet3["所在地"].tolist() 

url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

for i in range(len(df_sheet3)):
    q = df_sheet3["所在地"][i]
    res = requests.get(url + urllib.parse.quote(q))

    latitude_list.append(res.json()[0]["geometry"]["coordinates"][1]) 
    longtude_list.append(res.json()[0]["geometry"]["coordinates"][0])

#マップにプロット
map = folium.Map(location=[latitude_list[0],longtude_list[0]],zoom_start=15)

for i in range(len(df_sheet3)):
    popup_content = f"<div style='writing-mode: horizontal-tb; white-space: nowrap;'>"
    popup_content += f"{name_list[i]}<br>"
    popup_content += f"販売価格: {price_list[i]}<br>"
    popup_content += f"間取り: {plan_list[i]}<br>"
    popup_content += f"建物面積: {area_list[i]}<br>"
    popup_content += f'<a href={URL_list[i]} target="_blank">詳細</a>' 
    popup_content += "</div>"
    
    iframe = branca.element.IFrame(html=popup_content, width=300, height=160)
    popup = folium.Popup(iframe, max_width=300)

    folium.Marker(location=[latitude_list[i], longtude_list[i]], popup=popup,icon=folium.Icon(color='purple')).add_to(map)

map


In [45]:
df_sheet3

,物件名,販売価格,販売価格2,所在地,沿線・駅,面積,面積2,間取り,間取り2,バルコニー,築年月,URL
0,ハーモニーヒルズやすらぎの街【安心を買うなら、朝日土…,2980万円,2980,神奈川県横浜市都筑区加賀原１,グリーンライン「川和町」徒歩21分,76.93m2（壁芯）,76.93,3LDK,3,13.56m2,1995年8月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...
1,メゾン桜が丘【安心を買うなら、朝日土地建物へ】,3499万円,3499,神奈川県横浜市都筑区大丸,グリーンライン「都筑ふれあいの丘」徒歩8分,68.02m2（壁芯）,68.02,3LDK,3,10.66m2,1987年5月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...
2,クリオ横浜鴨居【安心を買うなら、朝日土地建物へ】,3780万円,3780,神奈川県横浜市都筑区池辺町,ＪＲ横浜線「鴨居」徒歩14分,65m2（壁芯）,65,3LDK,3,10.2m2,2011年2月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...
3,ダイアパレスつづきの丘,3780万円,3780,神奈川県横浜市都筑区池辺町,ＪＲ横浜線「鴨居」徒歩16分,63.62m2（壁芯）,63.62,3LDK,3,11.4m2,2000年3月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...
4,レフィーズ荏田 ～平成19年築の新耐震基準マンション…,3800万円,3800,神奈川県横浜市都筑区荏田南４,東急田園都市線「江田」徒歩17分,75.83m2（壁芯）,75.83,3LDK,3,18.94m2,2007年7月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...
5,コンフォール東山田【安心を買うなら、朝日土地建物へ】,3890万円,3890,神奈川県横浜市都筑区東山田４,グリーンライン「北山田」徒歩15分,104.62m2（壁芯）,104.62,4LDK,4,15.81m2,1993年12月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...
6,コンフォール東山田5-4号棟,3890万円,3890,神奈川県横浜市都筑区東山田４,グリーンライン「北山田」徒歩15分,104.62m2,104.62,4LDK,4,15.81m2,1993年12月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...
7,グレーシアセンター北,3980万円,3980,神奈川県横浜市都筑区南山田２-３９－１３,グリーンライン「北山田」徒歩13分,70.91m2（壁芯）,70.91,3LDK,3,9.76m2,1997年8月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...
8,クリオ鴨居,3980万円,3980,神奈川県横浜市都筑区佐江戸町,ＪＲ横浜線「鴨居」徒歩13分,70.16m2（壁芯）,70.16,2LDK,2,10.79m2,2012年2月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...
9,藤和ライブタウンセンター南 参番館,4180万円,4180,神奈川県横浜市都筑区大丸,グリーンライン「都筑ふれあいの丘」徒歩5分,70.14m2（壁芯）,70.14,3LDK,3,9m2,2000年2月,https://suumo.jp/ms/chuko/kanagawa/sc_yokohama...


In [41]:
sheet4 = file.worksheet('新築マンション')

# sheet4のデータフレームを作成
data_sheet4 = sheet4.get_all_values()
df_sheet4 = pd.DataFrame(data_sheet4[1:], columns=data_sheet4[0])

#新築マンションの緯度経度取得
latitude_list = [] 
longtude_list = []

name_list = df_sheet4["物件名"].tolist()
price_list = df_sheet4["販売価格"].tolist() 
plan_list = df_sheet4["間取り"].tolist()
URL_list = df_sheet4["URL"].tolist()
address_list = df_sheet4["所在地"].tolist() 

url = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="

for i in range(len(df_sheet4)):
    q = df_sheet4["所在地"][i]
    res = requests.get(url + urllib.parse.quote(q))

    latitude_list.append(res.json()[0]["geometry"]["coordinates"][1]) 
    longtude_list.append(res.json()[0]["geometry"]["coordinates"][0])

#マップにプロット
map = folium.Map(location=[latitude_list[0],longtude_list[0]],zoom_start=15)

for i in range(len(df_sheet4)):
    popup_content = f"<div style='writing-mode: horizontal-tb; white-space: nowrap;'>"
    popup_content += f"{name_list[i]}<br>"
    popup_content += f"販売価格: {price_list[i]}<br>"
    popup_content += f"間取り: {plan_list[i]}<br>"
    popup_content += f'<a href={URL_list[i]} target="_blank">詳細</a>' 
    popup_content += "</div>"
    
    iframe = branca.element.IFrame(html=popup_content, width=300, height=160)
    popup = folium.Popup(iframe, max_width=300)

    folium.Marker(location=[latitude_list[i], longtude_list[i]], popup=popup,icon=folium.Icon(color='gray')).add_to(map)

map

In [46]:
df_sheet4

,物件名,販売価格,販売価格2,所在地,交通,間取り,間取り2,面積,面積2,引渡時期,URL
0,リストレジデンスセンター南パークサイドテラス,6990万円～8380万円,6990～8380,横浜市都筑区荏田東２,ブルーライン/センター南 徒歩12分,3LDK～4LDK,3,70.3m2～83.28m2,70.3,2024年8月下旬予定,https://suumo.jp/ms/shinchiku/kanagawa/sc_yoko...
1,クリオ横濱センター北グランシック,価格未定,未定,横浜市都筑区牛久保西１,ブルーライン/センター北 徒歩9分,2LDK～3LDK+S（納戸）,2,80.11m2～132.55m2,80.11,2024年11月下旬予定,https://suumo.jp/ms/shinchiku/kanagawa/sc_yoko...
2,ザ・パークハウス 横浜川和町テラス,4728万円～5868万円,4728～5868,横浜市都筑区川和町,グリーンライン/川和町 徒歩3分,3LDK,3,71.82m2～79.8m2,71.82,2024年3月下旬予定,https://suumo.jp/ms/shinchiku/kanagawa/sc_yoko...
